# Завдання 1

## Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки Surprise.



In [1]:
from surprise import Dataset, accuracy
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from collections import defaultdict
from surprise import AlgoBase, Dataset, PredictionImpossible
import pandas as pd
import numpy as np

In [2]:
data = Dataset.load_builtin('ml-100k')


Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\Саша/.surprise_data/ml-100k


## Виведіть перші 5 рядків датасету

In [5]:
ratings = data.raw_ratings
df = pd.DataFrame(ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])
print(df.head())

  user_id item_id  rating  timestamp
0     196     242     3.0  881250949
1     186     302     3.0  891717742
2      22     377     1.0  878887116
3     244      51     2.0  880606923
4     166     346     1.0  886397596


##  Реалізуйте два алгоритми для рекомендаційної системи на основі цього датасету. Можна вибрати будь-які алгоритми з бібліотеки Surprise

In [6]:
train_set, test_set = train_test_split(data, test_size=0.2)

alg1 = SVD()
alg2 = KNNBasic()

##  Використайте крос-валідацію для підбору оптимальних параметрів для обох алгоритмів.

### Крос-валідація SVD


In [7]:
param_grid = {'n_epochs': [5, 10, 20], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}
gs_svd = GridSearchCV(SVD, param_grid, measures=['mae'], cv=5)
gs_svd.fit(data)
print("Найкращі параметри для SVD", gs_svd.best_params['mae'])

Найкращі параметри для SVD {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.4}


### Крос-валідація KNNBasic

In [8]:
param_grid = {'k': [20, 40, 60], 'sim_options': {'name': ['msd', 'cosine'], 'user_based': [True, False]}}
gs_knn = GridSearchCV(KNNBasic, param_grid, measures=['mae'], cv=5)
gs_knn.fit(data)
print("Найкращі параметри для KNNBasic", gs_knn.best_params['mae'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...


In [9]:
knn_best = gs_knn.best_estimator['mae']
svd_best = gs_svd.best_estimator['mae']
knn_result = cross_validate(knn_best, data, measures=['mae'], cv=5, verbose=False)
svd_result = cross_validate(svd_best, data, measures=['mae'], cv=5, verbose=False)
mae_knn = knn_result['test_mae'].mean()
mae_svd = svd_result['test_mae'].mean()
print(f"MAE для KNNBasic: {mae_knn}")
print(f"MAE для SVD: {mae_svd}")
alg_best = knn_best if mae_knn < mae_svd else svd_best
print(f"Найкращий алгоритм: {'KNNBasic' if mae_knn < mae_svd else 'SVD'}")

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
MAE для KNNBasic: 0.7707887630159491
MAE для SVD: 0.7659222343067705
Найкращий алгоритм: SVD


##  Виведи рекомендації (10 фільмів) для конкретного користувача.

In [13]:
from collections import defaultdict

def get_top_n_recommendations(predictions, n=10):
    top_n_recommendations = defaultdict(list)

    for user_id, item_id, true_rating, estimated_rating, _ in predictions:
        top_n_recommendations[user_id].append((item_id, estimated_rating))
    
    for user_id, user_ratings in top_n_recommendations.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n_recommendations[user_id] = user_ratings[:n]
    
    return top_n_recommendations

best_algorithm = gs_svd.best_estimator['mae']
full_train_set = data.build_full_train_set()
best_algorithm.fit(full_train_set)
anti_test_set = full_train_set.build_anti_testset()
predictions = best_algorithm.test(anti_test_set)
top_n_recommendations = get_top_n_recommendations(predictions, n=10)
user_id = '48'
print(f"Рекомендації користувача {user_id}:")
for item_id, estimated_rating in top_n_recommendations[user_id]:
    print(f"Фільм: {item_id}, Рейтинг: {estimated_rating:.2f}")

Рекомендації користувача 48:
Фільм: 408, Рейтинг: 4.27
Фільм: 169, Рейтинг: 4.24
Фільм: 318, Рейтинг: 4.23
Фільм: 64, Рейтинг: 4.20
Фільм: 114, Рейтинг: 4.17
Фільм: 12, Рейтинг: 4.16
Фільм: 1449, Рейтинг: 4.15
Фільм: 178, Рейтинг: 4.13
Фільм: 513, Рейтинг: 4.11
Фільм: 272, Рейтинг: 4.09


# Завдання 2

## Побудуйте власну рекомендаційну систему

In [22]:
import numpy as np
from surprise import Dataset, AlgoBase, PredictionImpossible
from surprise.model_selection import train_test_split, KFold
from surprise import accuracy

data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=0.3)

In [27]:
class CustomAlgorithm(AlgoBase):
    def __init__(self):
        super().__init__()

    def fit(self, trainset):
        super().fit(trainset)
        self.global_mean = np.mean([r for (_, _, r) in trainset.all_ratings()])
        self.trainset = trainset  
        return self

    def estimate(self, user_id, item_id):
        if not self.trainset.knows_user(user_id) or not self.trainset.knows_item(item_id):
            raise PredictionImpossible("Помилка пошуку. Спробуйте ще раз")
        return self.global_mean

def get_top_n_recommendations(algorithm, user_id, n=10):
    user_inner_id = algorithm.trainset.to_inner_uid(user_id)
    all_items = algorithm.trainset.all_items()
    
    recommendations = []
    for item in all_items:
        try:
            est = algorithm.estimate(user_inner_id, item)
            recommendations.append((item, est))
        except PredictionImpossible:
            continue

    recommendations.sort(key=lambda x: x[1], reverse=True)
    top_n = [algorithm.trainset.to_raw_iid(item_id) for item_id, _ in recommendations[:n]]
    
    return top_n
my_algorithm = CustomAlgorithm()
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    my_algorithm.fit(trainset)
    predictions = my_algorithm.test(testset)
    accuracy.mae(predictions)
user_id = str(48)
top_recommendations = get_top_n_recommendations(my_algorithm, user_id)
print("Топ 10 рекомендацій для користувача 48:", top_recommendations)

MAE:  0.9465
MAE:  0.9439
MAE:  0.9436
Топ 10 рекомендацій для користувача 48: ['501', '202', '53', '214', '663', '237', '117', '227', '175', '91']


## Висновки
## Програма реалізує просту рекомендаційну систему на основі алгоритму, що використовує середній рейтинг для прогнозування вподобань користувачів. Це дозволяє отримувати рекомендації для конкретного користувача